In [ ]:
import pandas as pd 
from pandas.tseries.holiday import get_calendar, HolidayCalendarFactory, Holiday,
import numpy as np 
import psycopg2
from datetime import datetime, timedelta
import warnings
warnings.filterwarning("ignore")

pd.set_option('display.max_columns',None)

from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm 
import matploatlib.pyplot as pyplot
from statsmodels.tsa.statespace.sarimax import sarimax
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_absoluate_error, mean_absoluate_percentage_error
from xgboost import xgbregressor
from sklearn.model_selection import cross_val_score, RepeatedKFold
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
#parameters setup and create forecast DF
train_start_date = '2020-01-03'
train_end_date = '2023-12-24'

future_prediction_start_date = '2023-12-31'
future_prediction_end_date = '2024-12-19'

fpsd = datetime.strptime(future_prediction_start_date, '%Y-%m-%d')
fped = datetime.strptime(future_prediction_end_date, '%Y-%m-%d')

#create prediction dataframe
index_of_fc = pd.date_range(fpsd, periods = n_periods, freq = 'W-SUN')

In [ ]:
#import actual data and regressors actuals

historical_data = []
regressor_historicals = []

In [ ]:
#split the dataset
def sample_data_split(data):

    n_test = int(len(data)*0.2)
    train = data[~data.index.isin(data.index.unique()[-n_test:])]
    test = data[~data.index.isin(data.index.unique()[-n_test:])]

    return n_test,train.sort_values(by = 'ship_week'),test.sort_values(by='ship_week')

In [ ]:
#set up sarima and xgboost models
def sarima_model(train,test,exog_df_fut,ar,i,ma,ar_s,i_s,ma_s,s):
    exog_df_future = exog_df_fut.copy()
    model =sarimax(train['var'],exog = train.iloc[:,1:],order=(ar,i,ma),seasonal_order=(ar_s,i_s,ma_s.s),enforce_invertibility=False)
    model_fit = model.fit(maxiter=1000,method='nm')
    print(model_fit.pvalues[model_pvalues<0.05].index)

    train['pred'] = model_fit.get_prediction(start = train.index[0], end = train.index[len(train)-1],\
                                              index = train.index\
                                              exog = pd.dataframe(train.iloc[:,1:]), \
                                              dynamic = False).predicted_mean
                                
    test['pred'] = model_fit.get_prediction(start = test.index[0], end = test.index[len(test)-1],\
                                            exog = pd.dataframe(test.iloc[:,1:]),\
                                            dynamic = False).predicted_mean

    future_df['pred'] = pred_series[test.shape[0]:]
    test_mape = mean_absolute_percentage_error(test['var'],test.pred)
    test_mae = mean_absolute_error(test['var'],test.pred)

    return (train,test,future_df,test_mape,test_mae,model)

def auto_arima_model(train_df,test_df,exog_df_fut,p_list,d_list,q_list,P_list,D_list,Q_list):
    train = train_df.copy()
    test = test_df.copy()
    exog_df_future = exog_df_fut.copy()
    a_df = pd.concat([train,test])
    model_fit = auto_arima(a_df['var'],X=a_df.iloc[:,1:],
                           start_p=p_list[0],max_p=p_list[1],max_d=d_list[0],
                           start_q=q_list[0],max_q=q_list[1],
                           start_P=P_list[0],max_P=P_list[1],max_D=D_list[0],
                           seasonal = True,
                           start_Q=Q_list[0],max_Q=Q_list[1],
                           maxiter=2000,scoring='mae',
                           out_of_sample_size=len(test)
                           )
    print(model_fit.summary())
    
    train['pred'],confint_train = model_fit.predict(start = a_df.index[0],end = a_df.index[len(a_df)-len(test)-1], \
                                                    n_periods=len(a_df)-len(test),X = a_df.iloc[:-len(test),1:],return_conf_int=True)
    train['confint_low'] = pd.series(confint_train[:,0], index=test.index)
    train['confint_high'] = pd.series(confint_train[:,1], index=test.index)
    

    test['pred'],confint_test = model_fit.predict(start = test.index[0], end = test.index[len(test)-1],\
                                                  n_periods=len(test),X = test.iloc[:,1:],return_conf_int=True)
    test['confint_low'] = pd.series(confint_test[:,0], index=test.index)
    test['confint_high'] = pd.series(confint_test[:,1], index=test.index)

    test_mae = mean_absolute_error(test['var'],test.pred)
    test_mape = mean_absolute_percentage_error(test['var'],test.pred)

    future_df = exog_df_future.copy()
    future_df['pred'],confint_fut = model_fit.predict(start = future_df.index[0], end = future_df.index[len(future_df)-1],\
                                                      n_peirods=len(future_df),X = future_df,return_conf_int = True)
    future_df['confint_low'] = pd.series(confint_fut[:, 0], index=future_df.index)
    future_df['confint_high'] = pd.series(confint_fut[:, 1], index=future_df.index)

    return (train, test, future_df, test_mape, test_mae, model_fit)

def fit_sarima_model(a_d, exog_df_fut):
    exog_df_future = exog_df_fut.copy()
    X = a_df['var'].values
    result = adfuller(X,autolag='AIC', regression='ct')
    print('ADF Statistic: %f' % results[0])
    print('p-value: %f' % results[1])
    print("Num Of Lags : ",% results[2])
    print('Critical Values:')
    
    for key, value in results[4].items():
        print('\t%s: %.3f' % (key,value))

    is_valid = 1
    if results[1] > 0.1
        print('Time series is not stationary, SARIMA is not recommended.')
        is_valid = 0

    n_sarima, train_df, test_df = sample_data_split(a_df)
    var_stat_mean = a_df['var'].mean()
    var_stat_std = a_df['var'].std()

    if var_stat_std !=0:
        sm.graphics.tsa.plot_acf(a_df['var'])
        plt.show()
        sm.graphics.tsa.plot_pacf(a_df['var'])
        plt.show()

        decompose_data = seasonal_decompose(a_df['var'], model="additive")
        decompose_data.plot()

        p_list = [0:15]
        d_list = [15]
        q_list = [0,15]
        P_list = [0:15]
        D_list = [15]
        Q_list = [0,15]

        train_df_aa, test_df_aa, fuuture_df_aa, test_mape_aa, test_mae_aa, model_aa = auto_arima_model(train_df,test_df,exog_df_future,p_list,\
                                                                                                     d_list,q_list_,P_list,D_list,Q_list)

        print('Auto Arima Model Error, MAPE', test_mape_aa,', MAE:',test_mae_aa)

        train_df_sa, test_df_sa, future_df_sa, test_mape_sa, test_mae_sa, model_sa = sarima_model(train_df,test_df,exog_df_future,0,1,0,0,0,0,52)
        print('SARIMAX Model Error, MAPE', test_mape_sa,', MAE:', test_mae_sa)

        if (test_mae_aa <= test_mae_sa):
            test_arima_pred = test_df_aa.copy()
            train_arima_pred = train_df_aa.copy()
            future_df_pred = future_df_aa.copy()
            best_model_mae = test_mae_aa
        else:
            test_arima_pred = test_df_aa.copy()
            train_arima_pred = train_df_aa.copy()
            future_df_pred = future_df_aa.copy()
            best_model_mae = test_mae_sa

    else: 
        train_arima_pred = train_df.copy()
        test_arima_pred = test_df.copy()
        train_arima_pred['pred'] = var_stat_mean
        test_arima_pred['pred'] = var_stat_mean
        best_model_mae = 0

    #Plot
    plt.figure(figsize=(15,7))
    plt.plot(a_df["var"], color='#1f76b4', label='actual')
    plt.plot(pd.concat([train_df_aa['pred'],test_df_aa['pred'],future_df_aa['pred']]), color='darkgreen' ,label="autoarima")
    plt.plot(pd.concat([train_df_sa['pred'],test_df_sa['pred'],future_df_sa['pred']]), color='red' ,label="sarima")
    plt.fill_between(pd.concat([a_df,future_df_aa]).index,
                    pd.concat([train_df_aa['confint_low'],test_df_aa['confint_ dlow'],future_df_aa['confint_low']]),
                    pd.concat([train_df_aa['confint_high'],test_df_aa['confint_high'],future_df_aa['confint_high']]),
                    color='k',alpha=.15,label="autoarima conf")

    return train_arima_pred,test_arima_pred,future_df_pred,is_valid,best_model_mae


def add_lags(df)
    target_map = df['var'].to_dict()
    df['lag1'] = (df.index - pd.Timedeltaa(53,"W")).map(target_map)
    return df

def add_lags_future(a_old_df,a_future_df):
    a_date = (a_future_df.index - pd.Timedelta(53,"W"))
    a_old_var = a_old_df.loc[a_date,'var']
    a_future_df['lag1'] = a_old_var.values
    return a_future_df

In [ ]:
#define functions

def ensemble_model(trasin_df_arima, train_df_xgb, test_df_arima, test_df_xgb, future_df_xg,is_valid,model_mae_arima,model_mae_xgb):
    #preprocess train dataset
    train_df_arima.rename(columns={'pred':'arima_pred'},inplace=True)
    train_df_xgb.rename(columns={'pred':'xgb_pred'},inplace=True)
    train_df_final = train_df_arima.copy()
    train_df_final['is_arima_valid'] = is_valid
    train_df_final['xgb_pred'] = train_df_xgb['xgb_pred']

    #preprocess test dataset
    test_df_arima.rename(columns={'pred':'arima_pred'},inplace=True)
    test_df_xgb.rename(columns={'pred':'xgb_pred'},inplace=True)
    test_df_final = test_df_arima.copy()
    test_df_final['is_arima_valid'] = is_valid
    test_df_final['xgb_pred'] = test_df_xgb['xgb_pred']

    #preprocess forecasting dataset
    future_df_arima.rename(columns={'pred':'arima_pred'},inplace=True)
    future_df_xgb.rename(columns={'pred':'xgb_pred'},inplace=True)
    future_df_final = future_df_arima.copy()
    future_df_final['is_arima_valid'] = is_valid
    future_df_final['xgb_pred'] = future_df_xgb['xgb_pred']

    #if
    if (model_mae_xgb <= model_mae_arima):
        train_df_final['best_pred'] = train_df_final['xgb_pred']
        test_df_final['best_pred'] = test_df_final['xgb_pred']
        future_df_final['best_pred'] = test_df_final['xgb_pred']
        best_mae = model_mae_xgb
        model_use = 'XGB'
    else:        
        train_df_final['best_pred'] = train_df_final['arima_pred']
        test_df_final['best_pred'] = test_df_final['arima_pred']
        future_df_final['best_pred'] = test_df_final['arima_pred']
        best_mae = model_mae_xgb
        model_use = 'ARIMA'

    train_df_final['model_used'] = model_used
    test_df_final['model_used'] = model_used
    future_df_final['model_used'] = model_used
    print('Best MAE: ',best_mae)
    
    return train_df_final,test_df_final,future_df_final,best mae
